### Polynomial Regression is a form of linear regression in which the relationship between the independent variable x and dependent variable y is not linear but it is the nth degree of polynomial.

In [1]:
from sklearn.preprocessing import PolynomialFeatures
from sklearn.linear_model import LinearRegression
from sklearn.metrics import mean_squared_error, r2_score

In [2]:
from scipy import stats
from sklearn.base import BaseEstimator
#from sklearn.grid_search import GridSearchCV
from sklearn.model_selection import learning_curve, GridSearchCV

In [3]:
import pandas as pd
import numpy as np
import pickle
import matplotlib.pyplot as plt
import operator

In [4]:
ocr_500=np.load('ocr_500.npy')
tm_500=np.load('tm_500.npy')
p2v2_500=np.load('p2v2_500.npy')
p2v_500=np.load('p2v_500.npy')
p2v3_500=np.load('p2v3_500.npy')
tm_p2v_500=np.load('tm_p2v_500.npy')
tm_p2v2_500=np.load('tm_p2v2_500.npy')
tm_p2v3_500=np.load('tm_p2v3_500.npy')
ocr_tm_500=np.load('ocr_tm_500.npy')
ocr_p2v_500=np.load('ocr_p2v_500.npy')
ocr_p2v2_500=np.load('ocr_p2v2_500.npy')
ocr_p2v3_500=np.load('ocr_p2v3_500.npy')
ocr_tm_p2v_500=np.load('ocr_tm_p2v_500.npy')
ocr_tm_p2v2_500=np.load('ocr_tm_p2v2_500.npy')
ocr_tm_p2v3_500=np.load('ocr_tm_p2v3_500.npy')
Combined_np1=np.load('Combined_np1.npy')
Combined_np2=np.load('Combined_np2.npy')
Combined_np3=np.load('Combined_np3.npy')

In [5]:
ocr_annox=np.load('ocr_annox.npy')
tm_annox=np.load('tm_annox.npy')
p2v_annox=np.load('p2v_annox.npy')
tm_p2v_annox=np.load('tm_p2v_annox.npy')
ocr_p2v_annox=np.load('ocr_p2v_annox.npy')
ocr_tm_annox=np.load('ocr_tm_annox.npy')

In [6]:
p2v2_annox=np.load('p2v2_annox.npy')
tm_p2v2_annox=np.load('tm_p2v2_annox.npy')
ocr_p2v2_annox=np.load('ocr_p2v2_annox.npy')
p2v3_annox=np.load('p2v3_annox.npy')
tm_p2v3_annox=np.load('tm_p2v3_annox.npy')
ocr_p2v3_annox=np.load('ocr_p2v3_annox.npy')

In [7]:
df=pd.read_pickle('df_p2v2.pkl')

In [11]:
df.head(10)

,ID,Images,OCR,Template_Name,dh_1,dh_2,dh_3,cln_content,ocr_emb,tm_emb,p2v2_emb,ocr_tm_p2v2_500_emb,porn_score,profanity_score,pronoun_count,polarity_score,sentiment,rating_avg,min_max
0,1,NaN,當我再差幾秒就遲到時，卻在這幾秒內衝到學校。我。學校。,自信的善逸,0,0,0,"[當我, 再, 差, 幾秒, 就, 遲到, 時, ，, 卻, 在, 這幾秒, 內, 衝到, ...","[-0.0034480316, -0.0014064792, -0.020086164, -...","[-0.051428244, -0.006179096, 0.012731423, -0.0...","[-0.5377346, 0.8174292, 0.43731427, -0.4928556...","[-0.0034480316, -0.0014064792, -0.020086164, -...",0,0,1,0,0,0.000000,0.000000
1,2,NaN,當你以為他們要幫你拿倉鼠飼料來餵你吃，但手上卻是拿著兩條蟒蛇加上一個攝影機。,怕爆倉鼠,0,2,0,"[當你以, 為, 他們, 要, 幫, 你, 拿, 倉鼠, 飼料, 來, 餵, 你, 吃, ，...","[0.013175973, -0.0027130025, 0.030018723, 0.02...","[-0.005815765, 0.007365296, 0.0742434, 0.01984...","[-0.35660374, 0.6002908, 0.49771008, -0.273855...","[0.013175973, -0.0027130025, 0.030018723, 0.02...",0,0,1,0,0,0.666667,0.222222
2,3,NaN,紅燈了行人大搖大擺的過馬路。行人。台灣的計程車司機\n,小丑被車撞,0,1,0,"[紅燈, 了, 行, 人大, 搖大, 擺, 的, 過馬路, 。, 行人, 。, 台灣, 的,...","[-0.035057098, -0.073808864, -0.018074743, 0.0...","[-0.028380213, -0.022930214, -0.0462088, 0.035...","[-0.20228152, 0.4813008, 0.08527977, -0.103984...","[-0.035057098, -0.073808864, -0.018074743, 0.0...",0,0,0,0,0,0.333333,0.111111
3,4,NaN,辛普森家庭從來不會讓我失望。AREA 51 WARNING:DO NOT ENTER!\n\n,51禁區辛普森家族早就知道了,0,0,0,"[辛普森, 家庭, 從來, 不會, 讓, 我, 失望, 。, AREA, , 51, ,...","[0.014597042, -0.008372392, 0.042043738, 0.007...","[0.03861497, -0.026871063, 0.057037357, 0.0161...","[0.18491076, 0.5516072, 0.264255, 0.2202262, -...","[0.014597042, -0.008372392, 0.042043738, 0.007...",0,0,1,-3,-1,0.000000,0.000000
4,5,NaN,武漢肺炎疫情日漸嚴重。習近平:,小熊維尼笑,0,1,2,"[武漢, 肺炎, 疫情, 日漸, 嚴重, 。, 習近, 平, :]","[0.0012484181, -0.026934076, 0.0077450513, 0.0...","[-0.017085267, -0.011113647, 0.05663415, 0.012...","[-0.13843994, 0.43964586, 0.3693608, -0.027635...","[0.0012484181, -0.026934076, 0.0077450513, 0.0...",0,0,0,-2,-1,1.000000,0.333333
5,6,NaN,"女人:我的傷勢如何?護理人員:還不錯,只有腿部擦傷而已。女人:我能看看嗎?護理人員:不太方便...",女人躺在救護車上,2,2,2,"[女人, :, 我, 的, 傷勢, 如何, ?, 護理, 人員, :, 還不錯, ,, 只有...","[0.051407035, 0.015403028, 0.06547234, 0.00215...","[0.018699622, -0.018211206, -0.0697222, 0.0294...","[-0.6882256, 0.49104884, 0.3950626, -0.0669471...","[0.051407035, 0.015403028, 0.06547234, 0.00215...",0,0,1,-2,-1,2.000000,0.666667
6,7,NaN,"醫生:史蒂芬妮,你快死了。病人:什麼!?醫生:愚人節快樂!病人:哈哈哈,可是今天不是四月一號...",醫生看著苦惱的病人,2,2,2,"[醫生, :, 史蒂芬, 妮, ,, 你, 快死了, 。, 病人, :, 什麼, !, ?,...","[0.006917947, -0.040207602, 0.0038061142, -0.0...","[0.019617623, -0.023825625, -0.020928422, 0.01...","[-0.52309585, 0.67755055, 0.43121225, -0.56486...","[0.006917947, -0.040207602, 0.0038061142, -0.0...",0,0,2,-4,-1,2.000000,0.666667
7,8,NaN,皮卡:我不是腦麻兒。皮卡。\n,低能兒,2,3,2,"[皮卡, :, 我, 不是, 腦麻兒, 。, 皮卡, 。, \n]","[0.06848506, -0.05862978, -0.0019475162, -0.03...","[0.013279542, -0.06721787, -0.024495786, -0.05...","[-0.45322442, 0.9036457, 0.34255874, -0.193167...","[0.06848506, -0.05862978, -0.0019475162, -0.03...",0,0,1,-1,-1,2.333333,0.777778
8,9,NaN,給三歲半的小妹妹買的芭比娃娃，但她的玩法好像和其他孩子有點不同..\n,芭比娃娃欺負別人,1,1,0,"[給三歲, 半, 的, 小妹妹, 買, 的, 芭比娃娃, ，, 但, 她, 的, 玩法, 好...","[0.032532975, 0.00016502145, 0.017045466, 0.01...","[-0.03801355, -0.023124255, 0.053622477, 0.006...","[-0.3106686, 0.575411, 0.7006884, -0.48054448,...","[0.032532975, 0.00016502145, 0.017045466, 0.01...",0,0,0,1,1,0.666667,0.222222
9,10,NaN,"中國人:「相信党,相信政府!」口罩都不用戴了?不戴!相信政府!",大媽開心、停屍間,2,2,2,"[中國人, :, 「, 相信, 党, ,, 相信, 政府, !, 」, 口罩, 都, 不用,...","[0.01874962, 0.055081934, 0.02705088, -0.02416...","[-0.006264757, -0.04730464, -0.008424059, -0.0...","[0.38690665, 0.57285905, 0.6802858, 0.15483646...","[0.01874962, 0.055081934, 0.02705088, -0.02416...",0,0,0,-1,-1,2.000000,0.666667


In [13]:
a=df['min_max']
a[335]

0.8888888888888888

In [63]:
x=ocr_annox
y= df.iloc[:,18].values

In [31]:
polynomial_features= PolynomialFeatures(degree=2)
x_poly = polynomial_features.fit_transform(x)

In [32]:
model = LinearRegression()
model.fit(x_poly, y)
y_poly_pred = model.predict(x_poly)

In [33]:
mse = mean_squared_error(y,y_poly_pred)
r2 = r2_score(y,y_poly_pred)
print(rmse)
print(r2)

2.349531116259377e-15
1.0


## Try Lasso and Ridge

In [14]:
#from sklearn.linear_model import Ridge
from sklearn.model_selection import cross_val_score
from sklearn.model_selection import train_test_split

In [15]:
x=ocr_tm_annox
y= df.iloc[:,18].values

In [18]:
x_train, x_test, y_train, y_test = train_test_split(x, y, test_size=0.3, random_state=0)

In [201]:
rr = Ridge(alpha=20)
rr.fit(x_train, y_train) 
pred_train_rr= rr.predict(x_train)
print(np.sqrt(mean_squared_error(y_train,pred_train_rr)))
print(r2_score(y_train, pred_train_rr))

pred_test_rr= rr.predict(x_test)
print(np.sqrt(mean_squared_error(y_test,pred_test_rr))) 
print(r2_score(y_test, pred_test_rr))

0.2258453533194777
0.3530848292755241
0.24024152191809162
0.2848335972318532


In [19]:
from sklearn.linear_model import Lasso

In [20]:
model_lasso = Lasso(alpha=1)
model_lasso.fit(x_train, y_train) 
pred_train_lasso= model_lasso.predict(x_train)
print(np.sqrt(mean_squared_error(y_train,pred_train_lasso)))
print(r2_score(y_train, pred_train_lasso))

pred_test_lasso= model_lasso.predict(x_test)
print(np.sqrt(mean_squared_error(y_test,pred_test_lasso))) 
print(r2_score(y_test, pred_test_lasso))

0.2807937853867435
0.0
0.28408252438056836
0.0


In [21]:
from sklearn.linear_model import LassoCV

In [22]:
#可以先試試
#LassoCV自動調節alpha可以實現選擇最佳的alpha
reg = LassoCV(cv=5, random_state=0).fit(x, y)
reg.score(x, y)

0.6494485986381049

In [16]:
loss=reg.mse_path_
print(loss.mean()) 

0.0643833147478511


In [24]:
pred_y=reg.predict(x)
dh_score=np.abs(pred_y)
dh_score[223]

0.5451526925680608

In [25]:
a=df['min_max']
a[223]

0.5555555555555556

In [38]:
# 另一種kfold作法
#from sklearn import model_selection
#from sklearn.linear_model import Lasso
#from sklearn.linear_model import LassoCV
seed = 1
kfold = model_selection.KFold(n_splits=5, random_state=seed)
model = LassoCV(random_state=0)
results = model_selection.cross_val_score(model, x, y, cv=kfold, scoring='r2')
loss = model_selection.cross_val_score(model, x, y, cv=kfold, scoring='neg_mean_squared_error')
print(results.mean())
print(loss.mean())

/home/jl8394/humor/lib/python3.6/site-packages/sklearn/model_selection/_split.py:296: FutureWarning: Setting a random_state has no effect since shuffle is False. This will raise an error in 0.24. You should leave random_state to its default (None), or set shuffle=True.
  FutureWarning
/home/jl8394/humor/lib/python3.6/site-packages/sklearn/linear_model/_coordinate_descent.py:476: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations. Duality gap: 0.00876227722072409, tolerance: 0.0025848726851851857
  positive)
/home/jl8394/humor/lib/python3.6/site-packages/sklearn/linear_model/_coordinate_descent.py:476: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations. Duality gap: 0.01761578919493534, tolerance: 0.0025848726851851857
  positive)
/home/jl8394/humor/lib/python3.6/site-packages/sklearn/linear_model/_coordinate_descent.py:476: ConvergenceWarning: Objective did not converge. You might want to 

/home/jl8394/humor/lib/python3.6/site-packages/sklearn/linear_model/_coordinate_descent.py:476: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations. Duality gap: 0.029649011191030278, tolerance: 0.0025848726851851857
  positive)
/home/jl8394/humor/lib/python3.6/site-packages/sklearn/linear_model/_coordinate_descent.py:476: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations. Duality gap: 0.012404189268787474, tolerance: 0.0025848726851851857
  positive)
/home/jl8394/humor/lib/python3.6/site-packages/sklearn/linear_model/_coordinate_descent.py:476: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations. Duality gap: 0.005298474308437051, tolerance: 0.0025848726851851857
  positive)
/home/jl8394/humor/lib/python3.6/site-packages/sklearn/linear_model/_coordinate_descent.py:476: ConvergenceWarning: Objective did not converge. You might want to increas

/home/jl8394/humor/lib/python3.6/site-packages/sklearn/linear_model/_coordinate_descent.py:476: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations. Duality gap: 0.006317436744447402, tolerance: 0.002569814814814815
  positive)
/home/jl8394/humor/lib/python3.6/site-packages/sklearn/linear_model/_coordinate_descent.py:476: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations. Duality gap: 0.024638783466419678, tolerance: 0.002569814814814815
  positive)
/home/jl8394/humor/lib/python3.6/site-packages/sklearn/linear_model/_coordinate_descent.py:476: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations. Duality gap: 0.014063877786259837, tolerance: 0.002569814814814815
  positive)
/home/jl8394/humor/lib/python3.6/site-packages/sklearn/linear_model/_coordinate_descent.py:476: ConvergenceWarning: Objective did not converge. You might want to increase t

/home/jl8394/humor/lib/python3.6/site-packages/sklearn/linear_model/_coordinate_descent.py:476: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations. Duality gap: 0.010000473291035525, tolerance: 0.002569814814814815
  positive)
/home/jl8394/humor/lib/python3.6/site-packages/sklearn/linear_model/_coordinate_descent.py:476: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations. Duality gap: 0.011127178638554128, tolerance: 0.002569814814814815
  positive)
/home/jl8394/humor/lib/python3.6/site-packages/sklearn/linear_model/_coordinate_descent.py:476: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations. Duality gap: 0.013389215010146012, tolerance: 0.002569814814814815
  positive)
/home/jl8394/humor/lib/python3.6/site-packages/sklearn/linear_model/_coordinate_descent.py:476: ConvergenceWarning: Objective did not converge. You might want to increase t

/home/jl8394/humor/lib/python3.6/site-packages/sklearn/linear_model/_coordinate_descent.py:476: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations. Duality gap: 0.012218588658708507, tolerance: 0.002553564814814815
  positive)
/home/jl8394/humor/lib/python3.6/site-packages/sklearn/linear_model/_coordinate_descent.py:476: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations. Duality gap: 0.01814304717859383, tolerance: 0.002553564814814815
  positive)
/home/jl8394/humor/lib/python3.6/site-packages/sklearn/linear_model/_coordinate_descent.py:476: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations. Duality gap: 0.019878594952708717, tolerance: 0.002553564814814815
  positive)
/home/jl8394/humor/lib/python3.6/site-packages/sklearn/linear_model/_coordinate_descent.py:476: ConvergenceWarning: Objective did not converge. You might want to increase th

...

/home/jl8394/humor/lib/python3.6/site-packages/sklearn/linear_model/_coordinate_descent.py:476: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations. Duality gap: 0.010252729778867462, tolerance: 0.002481712962962963
  positive)
/home/jl8394/humor/lib/python3.6/site-packages/sklearn/linear_model/_coordinate_descent.py:476: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations. Duality gap: 0.00920719011608323, tolerance: 0.002481712962962963
  positive)
/home/jl8394/humor/lib/python3.6/site-packages/sklearn/linear_model/_coordinate_descent.py:476: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations. Duality gap: 0.009707112078244862, tolerance: 0.002481712962962963
  positive)
/home/jl8394/humor/lib/python3.6/site-packages/sklearn/linear_model/_coordinate_descent.py:476: ConvergenceWarning: Objective did not converge. You might want to increase th

/home/jl8394/humor/lib/python3.6/site-packages/sklearn/linear_model/_coordinate_descent.py:476: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations. Duality gap: 0.006945407305996715, tolerance: 0.002381940586419753
  positive)
/home/jl8394/humor/lib/python3.6/site-packages/sklearn/linear_model/_coordinate_descent.py:476: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations. Duality gap: 0.004965585353674626, tolerance: 0.002381940586419753
  positive)
/home/jl8394/humor/lib/python3.6/site-packages/sklearn/linear_model/_coordinate_descent.py:476: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations. Duality gap: 0.0066252911834371875, tolerance: 0.002381940586419753
  positive)
/home/jl8394/humor/lib/python3.6/site-packages/sklearn/linear_model/_coordinate_descent.py:476: ConvergenceWarning: Objective did not converge. You might want to increase 

/home/jl8394/humor/lib/python3.6/site-packages/sklearn/linear_model/_coordinate_descent.py:476: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations. Duality gap: 0.02372741726416583, tolerance: 0.002477469135802469
  positive)
/home/jl8394/humor/lib/python3.6/site-packages/sklearn/linear_model/_coordinate_descent.py:476: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations. Duality gap: 0.026246992603171315, tolerance: 0.002477469135802469
  positive)
/home/jl8394/humor/lib/python3.6/site-packages/sklearn/linear_model/_coordinate_descent.py:476: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations. Duality gap: 0.03238353065429017, tolerance: 0.002477469135802469
  positive)
/home/jl8394/humor/lib/python3.6/site-packages/sklearn/linear_model/_coordinate_descent.py:476: ConvergenceWarning: Objective did not converge. You might want to increase the

0.15377394524037896
-0.06685072546211354


/home/jl8394/humor/lib/python3.6/site-packages/sklearn/linear_model/_coordinate_descent.py:476: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations. Duality gap: 0.009860886125642721, tolerance: 0.002477469135802469
  positive)


In [88]:
lassocv_alpha = reg.alpha_
print('lassocv_alpha',lassocv_alpha )

lassocv_alpha 0.00048683137852765005


## GridSearch

In [35]:
alpha = [1e-15, 1e-10, 1e-8, 1e-4, 1e-3,1e-2, 1, 5, 10, 20]

ridge = Ridge()

parameters = {'alpha': [1e-15, 1e-10, 1e-8, 1e-4, 1e-3,1e-2, 1, 5, 10, 20]}

ridge_regressor = GridSearchCV(ridge, parameters,scoring='r2', cv=5)

ridge_regressor.fit(x, y)

/home/jl8394/humor/lib/python3.6/site-packages/sklearn/linear_model/_ridge.py:190: UserWarning: Singular matrix in solving dual problem. Using least-squares solution instead.
  warnings.warn("Singular matrix in solving dual problem. Using "
/home/jl8394/humor/lib/python3.6/site-packages/sklearn/linear_model/_ridge.py:190: UserWarning: Singular matrix in solving dual problem. Using least-squares solution instead.
  warnings.warn("Singular matrix in solving dual problem. Using "
/home/jl8394/humor/lib/python3.6/site-packages/sklearn/linear_model/_ridge.py:188: LinAlgWarning: Ill-conditioned matrix (rcond=7.29727e-18): result may not be accurate.
  overwrite_a=False)
/home/jl8394/humor/lib/python3.6/site-packages/sklearn/linear_model/_ridge.py:188: LinAlgWarning: Ill-conditioned matrix (rcond=4.24923e-18): result may not be accurate.
  overwrite_a=False)
/home/jl8394/humor/lib/python3.6/site-packages/sklearn/linear_model/_ridge.py:190: UserWarning: Singular matrix in solving dual problem.

GridSearchCV(cv=5, error_score=nan,
             estimator=Ridge(alpha=1.0, copy_X=True, fit_intercept=True,
                             max_iter=None, normalize=False, random_state=None,
                             solver='auto', tol=0.001),
             iid='deprecated', n_jobs=None,
             param_grid={'alpha': [1e-15, 1e-10, 1e-08, 0.0001, 0.001, 0.01, 1,
                                   5, 10, 20]},
             pre_dispatch='2*n_jobs', refit=True, return_train_score=False,
             scoring='r2', verbose=0)

In [36]:
ridge_regressor.best_params_

{'alpha': 20}

In [37]:
ridge_regressor.best_score_

-2.41225133517124

In [38]:
cross_val_score(Ridge(), x, y, scoring="r2", cv = 5)

array([-2.17280729, -4.53751371, -8.82553373, -3.19655718, -2.66272453])